In [1]:
import json
import uuid

In [12]:
pip install kafka

Note: you may need to restart the kernel to use updated packages.


In [14]:
#following guide here for set up https://aiven-io.medium.com/teach-yourself-apache-kafka-and-python-with-a-jupyter-notebook-f0cfe006aad6

In [10]:
mkdir -p kafka-jupyter 

In [11]:
cd kafka-jupyter

C:\Users\spashtunyar\Documents\School\dsc650\dsc650\assignments\assignment08\kafka-jupyter


In [15]:
docker run                  \
--rm -p 8888:8888           \
-e JUPYTER_ENABLE_LAB=yes   \
-v "$PWD":/home/Shaq/work \
jupyter/datascience-notebook

SyntaxError: invalid syntax (856543572.py, line 1)

In [17]:
from kafka import KafkaProducer, KafkaAdminClient, KafkaConsumer, TopicPartition

SyntaxError: invalid syntax (simple.py, line 54)

In [7]:
from kafka import KafkaProducer, KafkaAdminClient

from kafka.errors import TopicAlreadyExistsError

SyntaxError: invalid syntax (simple.py, line 54)

In [8]:
from kafka.admin.new_topic import NewTopic

SyntaxError: invalid syntax (simple.py, line 54)

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [19]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Shaq',
    last_name='Pash'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Shaq',
 'last_name': 'Pash',
 'client_id': 'PashShaq',
 'topic_prefix': 'PashShaq'}

### Create Topic Utility Function

The `create_kafka_topic` helps create a Kafka topic based on your configuration settings.  For instance, if your first name is *John* and your last name is *Doe*, `create_kafka_topic('locations')` will create a topic with the name `DoeJohn-locations`.  The function will not create the topic if it already exists. 

In [20]:
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))
    
create_kafka_topic('locations')

NameError: name 'KafkaAdminClient' is not defined

### Kafka Producer

The following code creates a `KafkaProducer` object which you can use to send Python objects that are serialized as JSON.

**Note:** This producer serializes Python objects as JSON. This means that object must be JSON serializable.  As an example, Python `DateTime` values are not JSON serializable and must be converted to a string (e.g. ISO 8601) or a numeric value (e.g. a Unix timestamp) before being sent.

In [4]:
producer = KafkaProducer(
  bootstrap_servers=config['bootstrap_servers'],
  value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

### Send Data Function

The `send_data` function sends a Python object to a Kafka topic. This function adds the `topic_prefix` to the topic so `send_data('locations', data)` sends a JSON serialized message to `DoeJohn-locations`. The function also registers callbacks to let you know if the message has been sent or if an error has occured. 

In [8]:
def on_send_success(record_metadata):
    print('Message sent:\n    Topic: "{}"\n    Partition: {}\n    Offset: {}'.format(
        record_metadata.topic,
        record_metadata.partition,
        record_metadata.offset
    ))
    
def on_send_error(excp):
    print('I am an errback', exc_info=excp)
    # handle exception

def send_data(topic, data, config=config, producer=producer, msg_key=None):
    topic_prefix = config['topic_prefix']
    topic_name = '{}-{}'.format(topic_prefix, topic)
    
    if msg_key is not None:
        key = msg_key
    else:
        key = uuid.uuid4().hex
    
    producer.send(
        topic_name, 
        value=data,
        key=key.encode('utf-8')
    ).add_callback(on_send_success).add_errback(on_send_error)

In [9]:
example_data = dict(
    key1='value1',
    key2='value2'
)

send_data('locations', example_data)

Message sent:
    Topic: "DoeJohn-locations"
    Partition: 0
    Offset: 13
